## AI + 피지컬 컴퓨팅: BrainAI Car 자율주행 프로젝트 
### BrainAI Car 자율주행 [도로인식 AI 모델 최적화] 실행 프로그램

#### 실행 파일
- 실행 파일: Step6. BrainAI_Car_AD_배포 (모델 최적화).ipynb
- 필요 모듈: 없음

활동 제목: BrainAI Car 도로인식 자율주행 배포 (OpenVINO로 최적화) <br>
활동 설명: BrainAI Car의 도로인식 자율주행 AI 모델을  <br>
          자율 주행에 사용될 하드웨어에 최적화 시킵니다.
#### 주요 기능:
- Keras 모델을 불러와 OpenVINO로 최적화 시켜서 저장
- OpenVINO 모델과 Keras 모델의 모델 속도를 비교합니다.

#### 실행 방법:
1. 셀을 위에서부터 차례대로 실행
2. 각 셀의 설명을 읽고 이해하기
3. 결과를 확인하며 진행하기

In [1]:
import time
from pathlib import Path

import numpy as np
import openvino as ov
import tensorflow as tf

In [2]:
# 1단계: 경로 설정
print("\n" + "=" * 60)
print("🚗 BrainAI Car 모델 최적화 시작!")
print("=" * 60)

# 모델 파일 경로
KERAS_MODEL = "models_laneD1/brainai_car_laneD1_mobilenet_final.keras"
OPENVINO_MODEL = "models/laneD1/laneD1.xml"

# 저장 폴더 만들기
Path("models/laneD1").mkdir(parents=True, exist_ok=True)

print(f"\n📂 원본 모델: {KERAS_MODEL}")
print(f"📂 최적화 모델: {OPENVINO_MODEL}")


🚗 BrainAI Car 모델 최적화 시작!

📂 원본 모델: models_laneD1/brainai_car_laneD1_mobilenet_final.keras
📂 최적화 모델: models/laneD1/laneD1.xml


In [3]:
# 2단계: 원본 모델 불러오기
print("\n" + "=" * 60)
print("📥 원본 모델 불러오는 중...")
print("=" * 60)

try:
    model = tf.keras.models.load_model(KERAS_MODEL)
    print("✅ 모델 불러오기 성공!")
    print(f"   입력 크기: {model.input_shape}")
    print(f"   출력 크기: {model.output_shape}")
except FileNotFoundError:
    print("❌ 모델을 찾을 수 없습니다!")
    print("💡 먼저 모델 훈련을 완료하세요.")
    exit()


📥 원본 모델 불러오는 중...
✅ 모델 불러오기 성공!
   입력 크기: (None, 224, 224, 3)
   출력 크기: (None, 1)


In [4]:
# 3단계: OpenVINO로 변환하기
print("\n" + "=" * 60)
print("🔄 모델을 더 빠르게 만드는 중... (OpenVINO 변환)")
print("=" * 60)

# 테스트용 입력 데이터 만들기 (사진 1장 크기)
test_input = tf.zeros((1, 224, 224, 3), dtype=tf.float32)

# OpenVINO로 변환
ov_model = ov.convert_model(model, example_input=test_input)
print("✅ 변환 완료!")


🔄 모델을 더 빠르게 만드는 중... (OpenVINO 변환)



[ WARNING ]  From %s: The name %s is deprecated. Please use %s instead.



✅ 변환 완료!


In [5]:
# 4단계: 변환된 모델 저장하기
print("\n" + "=" * 60)
print("💾 최적화된 모델 저장 중...")
print("=" * 60)

ov.save_model(ov_model, OPENVINO_MODEL)

# 파일 크기 확인
xml_path = Path(OPENVINO_MODEL)
bin_path = Path(OPENVINO_MODEL.replace('.xml', '.bin'))

xml_size = xml_path.stat().st_size / (1024 * 1024)
bin_size = bin_path.stat().st_size / (1024 * 1024)

print(f"✅ 저장 완료!")
print(f"\n📊 모델 크기: {xml_size + bin_size:.2f} MB")


💾 최적화된 모델 저장 중...
✅ 저장 완료!

📊 모델 크기: 3.74 MB


In [7]:
# 6단계: 속도 비교하기

# OpenVINO로 모델 실행 준비
core = ov.Core()
compiled_model = core.compile_model(OPENVINO_MODEL, "CPU")

# 테스트 준비
test_data = np.random.randn(1, 224, 224, 3).astype(np.float32)
num_tests = 100

# 원본 모델 속도 측정
print(f"\n🔹 원본 모델 테스트 ({num_tests}회)...")
start = time.time()
for _ in range(num_tests):
    model.predict(test_data, verbose=0)
original_time = time.time() - start
print(f"   걸린 시간: {original_time:.2f}초")
print(f"   1회 평균: {original_time/num_tests*1000:.2f}ms")

# 최적화 모델 속도 측정
print(f"\n🔹 최적화 모델 테스트 ({num_tests}회)...")
start = time.time()
for _ in range(num_tests):
    compiled_model([test_data])
optimized_time = time.time() - start
print(f"   걸린 시간: {optimized_time:.2f}초")
print(f"   1회 평균: {optimized_time/num_tests*1000:.2f}ms")

# 결과 비교
speedup = original_time / optimized_time
improvement = (1 - optimized_time / original_time) * 100

print("\n" + "=" * 60)
print("📊 비교 결과")
print("=" * 60)
print(f"🚀 속도 향상: {speedup:.2f}배 빨라졌어요!")
print(f"⏰ 시간 단축: {improvement:.1f}%")


# 완료!
print("\n" + "=" * 60)
print("✅ 모델 최적화 완료!")
print("=" * 60)
print("\n💡 이제 이 모델을 사용하면:")
print("   • 더 빠르게 동작해요")
print("\n🎯 다음 단계:")
print("   1. BrainAI Car에 모델 업로드")
print("   2. 실제 주행 테스트")
print("=" * 60)


🔹 원본 모델 테스트 (100회)...
   걸린 시간: 15.16초
   1회 평균: 151.61ms

🔹 최적화 모델 테스트 (100회)...
   걸린 시간: 0.50초
   1회 평균: 5.04ms

📊 비교 결과
🚀 속도 향상: 30.07배 빨라졌어요!
⏰ 시간 단축: 96.7%

✅ 모델 최적화 완료!

💡 이제 이 모델을 사용하면:
   • 더 빠르게 동작해요
   • 라즈베리파이 같은 작은 컴퓨터에서도 잘 돌아가요
   • 배터리를 덜 사용해요

🎯 다음 단계:
   1. BrainAI Car에 모델 업로드
   2. 실제 주행 테스트
